# Generate Events

### Imports

In [5]:
from __future__ import absolute_import, division, print_function, unicode_literals

import logging
import numpy as np
import matplotlib
from matplotlib import pyplot as plt

from madminer.core import MadMiner
from madminer.delphes import DelphesReader
from madminer.sampling import combine_and_shuffle
from madminer.plotting import plot_distributions


# MadMiner output
logging.basicConfig(
    format='%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s',
    datefmt='%H:%M',
    level=logging.INFO
)

# Output of all other modules (e.g. matplotlib)
for key in logging.Logger.manager.loggerDict:
    if "madminer" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)

## Madgraph

### Signal

We use Madgraph to generate events at all benchmark points.

Due to the large number of benchmark points, the file sizes get pretty ridiculous. For ~120 benchmark points, you will need roughly 120 GB of space in the mg_process_directory. 

This cell will take roughly 1-2 days to complete.

In [ ]:
# Be sure to change these to the directory of the Madgraph installation.
###########################################################################################
mg_dir = '/home/kwl/Documents/hep/MG5_aMC_v2_6_5'    # Points to Madgraph Install     
events_dir = '/udd/madminer_events_uuuu/'            # Points to directory to save events 
                                                     # (where the run_XX folders are saved)
###########################################################################################
miner = MadMiner()
miner.load("meta/setup.h5")

miner.run_multiple(
    sample_benchmarks=None,
    mg_directory=mg_dir,
    mg_process_directory=events_dir+'signal_pythia',
    proc_card_file='cards/proc_card_signal.dat',
    param_card_template_file='cards/param_card.dat',
    pythia8_card_file='cards/pythia8_card.dat',
    run_card_files=['cards/run_card_signal.dat'],
    log_directory='logs/signal'
)

17:39 madminer.core        INFO    Found 2 parameters:
17:39 madminer.core        INFO       mzp (LHA: mass 56, maximal power in squared ME: (0,), range: (275.0, 325.0))
17:39 madminer.core        INFO       gzp (LHA: dminputs 2, maximal power in squared ME: (0,), range: (0.0, 5.0))
17:39 madminer.core        INFO    Found 121 benchmarks:
17:39 madminer.core        INFO       benchmark_0: mzp = 2.75e+02, gzp = 0.00e+00
17:39 madminer.core        INFO       benchmark_1: mzp = 2.75e+02, gzp = 0.50
17:39 madminer.core        INFO       benchmark_2: mzp = 2.75e+02, gzp = 1.00
17:39 madminer.core        INFO       benchmark_3: mzp = 2.75e+02, gzp = 1.50
17:39 madminer.core        INFO       benchmark_4: mzp = 2.75e+02, gzp = 2.00
17:39 madminer.core        INFO       benchmark_5: mzp = 2.75e+02, gzp = 2.50
17:39 madminer.core        INFO       benchmark_6: mzp = 2.75e+02, gzp = 3.00
17:39 madminer.core        INFO       benchmark_7: mzp = 2.75e+02, gzp = 3.50
17:39 madminer.core        INFO

17:39 madminer.core        INFO       benchmark_100: mzp = 3.20e+02, gzp = 0.50
17:39 madminer.core        INFO       benchmark_101: mzp = 3.20e+02, gzp = 1.00
17:39 madminer.core        INFO       benchmark_102: mzp = 3.20e+02, gzp = 1.50
17:39 madminer.core        INFO       benchmark_103: mzp = 3.20e+02, gzp = 2.00
17:39 madminer.core        INFO       benchmark_104: mzp = 3.20e+02, gzp = 2.50
17:39 madminer.core        INFO       benchmark_105: mzp = 3.20e+02, gzp = 3.00
17:39 madminer.core        INFO       benchmark_106: mzp = 3.20e+02, gzp = 3.50
17:39 madminer.core        INFO       benchmark_107: mzp = 3.20e+02, gzp = 4.00
17:39 madminer.core        INFO       benchmark_108: mzp = 3.20e+02, gzp = 4.50
17:39 madminer.core        INFO       benchmark_109: mzp = 3.20e+02, gzp = 5.00
17:39 madminer.core        INFO       benchmark_110: mzp = 3.25e+02, gzp = 0.00e+00
17:39 madminer.core        INFO       benchmark_111: mzp = 3.25e+02, gzp = 0.50
17:39 madminer.core        INFO     

### Background

In [2]:
miner.run(
    is_background=True,
    sample_benchmark='benchmark_0',
    mg_directory=mg_dir,
    mg_process_directory=events_dir + 'background_pythia',
    proc_card_file='cards/proc_card_background.dat',
    pythia8_card_file='cards/pythia8_card.dat',
    param_card_template_file='cards/param_card.dat',
    run_card_file='cards/run_card_background.dat',
    log_directory='logs/background',
)

u"\nminer.run(\n    is_background=True,\n    sample_benchmark='benchmark_0',\n    mg_directory=mg_dir,\n    mg_process_directory='/udd/mg_processes/background_pythia',\n    proc_card_file='cards/proc_card_background.dat',\n    pythia8_card_file='cards/pythia8_card.dat',\n    param_card_template_file='cards/param_card_template.dat',\n    run_card_file='cards/run_card_background.dat',\n    log_directory='logs/background',\n)\n"

## Delphes

### Stage events

In [ ]:
delphes = DelphesReader('meta/setup.h5')
for i in range(len(miner.benchmarks.keys())):
    run_name=str(i+1)
    if i < 9:
        run_name = '0' + str(i+1)
    delphes.add_sample(
        lhe_filename=events_dir + 'pythia_signal/run_' + run_name + '/unweighted_events.lhe.gz',
        hepmc_filename=events_dir + 'pythia_signal/run_' + run_name + '/tag_1_pythia8_events.hepmc.gz',
        sampled_from_benchmark=miner.benchmarks.keys()[i],
        is_background=False
    )

delphes.add_sample(
    lhe_filename=events_dir + 'pythia_background/run_01/unweighted_events.lhe.gz',
    hepmc_filename=events_dir + 'pythia_background/run_01/tag_1_pythia8_events.hepmc.gz',
    sampled_from_benchmark='benchmark_0',
    is_background=True
)

### Run Detector

In [ ]:
delphes.run_delphes(
    delphes_directory=mg_dir + '/Delphes',
    delphes_card='cards/delphes_card.dat',
    log_file='logs/delphes.log',
)

### Save detector level data

We have 8 features, which are the 2 leading jet four-momenta:

leading jet: $[\eta_0, \phi_0, p_{t,0}, m_0]$

subleading jet: $[\eta_1, \phi_1, p_{t,1}, m_1]$

We cut the invariant mass so that $m_{jj}=\sqrt{2 |p_{t,0}| |p_{t,1}| \left(\cosh(\eta_0 - \eta_1) - \cos(\phi_0 - \phi_1)\right)}\in[200, 400]~$GeV

In [ ]:
#### Observables #####

delphes.add_observable(
    'eta0',
    'j[0].eta',
    required=True
)
delphes.add_observable(
    'phi0',
    'j[0].phi()',
    required=True,
)
delphes.add_observable(
    'pt0',
    'j[0].pt',
    required=True,
)
delphes.add_observable(
    'm0',
    'j[0].m',
    required=True,
)
delphes.add_observable(
    'eta1',
    'j[1].eta',
    required=True,
)
delphes.add_observable(
    'phi1',
    'j[1].phi()',
    required=True,
)
delphes.add_observable(
    'pt1',
    'j[1].pt',
    required=True,
)
delphes.add_observable(
    'm1',
    'j[1].m',
    required=True,
)

#### Cuts ####

delphes.add_cut('np.sqrt(2 * np.abs(j[0].pt) * np.abs(j[1].pt) * (np.cosh(j[0].eta-j[1].eta)-np.cos(j[0].phi()-j[1].phi()))) > 200')
delphes.add_cut('np.sqrt(2 * np.abs(j[0].pt) * np.abs(j[1].pt) * (np.cosh(j[0].eta-j[1].eta)-np.cos(j[0].phi()-j[1].phi()))) < 400')

#### Process & Export ####

delphes.analyse_delphes_samples()
delphes.save('meta/delphes_data.h5')
combine_and_shuffle(
    ['data/delphes_data.h5'],
    'data/delphes_data_shuffled.h5'
)